In [ ]:
from Stochastic_Grid_world import GridWorld,standard_sticky_GW
from Stochastic_Agent import Agent

from numpy import random
from random import randint
from pprint import pprint


In [ ]:

def choose_action(distro: dict, epsilon=0):
    
    if randint(0,100) < epsilon :
        # use exploit/explore methods to find the best action in long term.
        pass
    
    acts  = list(distro.keys())
    probs = list(distro.values())
    
    index_list = [ i for i in range(len(acts))]
    index = random.choice(index_list,p=probs)
    
    return acts[index]


# assume without exploration implementation.
def play_game( grid_world:GridWorld, policy:dict ):
    
    states  = []
    rewards = {}
    actions = set()
    
    current_s = grid_world.current_state
    while not grid_world.game_over() :
        
        action_distro = policy.get(current_s)                 
        act = choose_action( action_distro )        # Deciding to choose an action
        rew = grid_world.transition(act)            # Perform an action and receive a reward.
        
        states.append(current_s)
        actions.add(act)
        
        # Reward is for the state which we moved to it, not for prev state.
        current_s = grid_world.current_state
        rewards.update({current_s:rew})
        
    
    # Adding the terminal state -> fixing terminal nighbor cells.
    states.append(grid_world.current_state)
    actions = list(actions)
    
    return states, actions, rewards


# model base 
def model_data( grid_world:GridWorld ):
    
    states  = grid_world.states
    rewards = grid_world.rewards
    actions = set()
    
    for state in grid_world.actions:
        for act in grid_world.actions.get(state,[]):
            actions.add(act)
    
    
    return states, actions, rewards

In [ ]:
env = standard_sticky_GW()
agent = Agent()

agent.initialize_Vs( env.rows, env.cols )

agent.policy = {
    (0,0):{"R":1.0}, 
    (0,1):{"R":1.0},
    (0,2):{"R":1.0},
    (1,0):{"U":1.0},
    (1,2):{"U":1.0},
    
    (2,0):{ "U":0.49, "R":0.51 },    # starting state in standard world.
    
    (2,1):{"R":1.0},
    (2,2):{"U":1.0},
    (2,3):{"L":1.0} 
}

In [ ]:
# Model-base info gathering
states, actions, rewards = model_data(env) 

# Setting Game details -> Agent
agent.exper_states  = states
agent.exper_actions = actions
agent.exper_rewards = rewards

# updating value function
agent.Update_value(env.probability)


# Logging
print("====== Final Value function ======")
pprint(agent.value_state_function)